In [11]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
import xesmf as xe
import regionmask
from tqdm import tqdm

import xarray as xr
import dask
dask.config.set(**{'array.slicing.split_large_chunks': True})
import sys
sys.path.insert(0, '/net/fs11/d0/emfreese/BC-IRF/')
import utils


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
pi = np.pi
rearth = 6.37122e6 
area_earth = 4*pi*rearth**2


In [2]:
years = utils.years

country_emit = 'INDONESIA'

## Add time dimension in terms of days
length_simulation = years
time_array = np.arange(0, years)

## Days per season
season_days = {'DJF': 90, 'MAM':92, 'JJA':92, 'SON':91}



In [3]:

## import the china global powerplant database from Springer et al.
CGP_df = pd.read_csv(f'{utils.data_output_path}plants/BC_SE_Asia_all_financing_SEA_GAINS_Springer.csv')


In [4]:
#CGP_df = CGP_df.loc[[1,2]]

In [12]:
def geocode_location(location_info, geolocator, delay=1.0, timeout=10, retries=3):
    """
    Geocode a location string with retry logic and extended timeout
    
    Parameters:
    -----------
    location_info : str
        Location string to geocode
    geolocator : Nominatim
        Geocoder instance
    delay : float
        Time in seconds to sleep between retries
    timeout : int
        Timeout in seconds for each request
    retries : int
        Number of times to retry if request fails
    """
    for attempt in range(retries):
        try:
            # Increase the timeout parameter to avoid timeouts
            location = geolocator.geocode(location_info, timeout=timeout)
            if location:
                return location.latitude, location.longitude
            time.sleep(delay)
            return None, None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            if attempt < retries - 1:
                print(f"Attempt {attempt+1} failed for {location_info}. Retrying after delay...")
                time.sleep(delay * (attempt + 1))  # Exponential backoff
            else:
                print(f"Failed to geocode {location_info} after {retries} attempts: {e}")
                return None, None
        except Exception as e:
            print(f"Geocoding error for {location_info}: {e}")
            return None, None
    
    return None, None

# Step 1: Create a unique identifier for each city-state-country combination
CGP_df['location_key'] = CGP_df.apply(
    lambda row: f"{row.get('CITY', '')}__{row.get('STATE', '')}__{row.get('COUNTRY', '')}", 
    axis=1
)

# Step 2: Get unique location combinations to geocode
unique_locations = CGP_df[['location_key', 'CITY', 'STATE', 'COUNTRY']].drop_duplicates()
print(f"Found {len(unique_locations)} unique locations to geocode (instead of {len(CGP_df)} individual rows)")

# Step 3: Create a mapping dictionary with geocoded coordinates
location_coords = {}
geolocator = Nominatim(user_agent="BC-IRF-app")  # Changed user agent slightly

# Hardcoded coordinates for common locations we're seeing timeouts for
hardcoded_locations = {}
#     "INDONESIA": (-0.7893, 113.9213),  # Center of Indonesia
#     "South Kalimantan, INDONESIA": (-3.0926, 115.2838),  # South Kalimantan province
#     "Tanjung - Kab Tabalong, South Kalimantan, INDONESIA": (-2.1552, 115.3963),  # Tabalong regency
#     "Kuching, Sarawak, MALAYSIA": (1.5497, 110.3626)  # Kuching
# }

for idx, row in tqdm(unique_locations.iterrows(), total=len(unique_locations), desc="Geocoding unique locations"):
    location_key = row['location_key']
    
    # First check if we have hardcoded coordinates for this location
    location_string = f"{row.get('CITY', '')}, {row.get('STATE', '')}, {row.get('COUNTRY', '')}"
    if location_string in hardcoded_locations:
        lat, lon = hardcoded_locations[location_string]
        location_coords[location_key] = (lat, lon)
        continue
    
    # Otherwise proceed with the regular geocoding paths
    # Try with city, state, country
    if pd.notna(row['CITY']) and pd.notna(row['STATE']) and pd.notna(row['COUNTRY']):
        location_info = f"{row['CITY']}, {row['STATE']}, {row['COUNTRY']}"
        lat, lon = geocode_location(location_info, geolocator, delay=2.0, timeout=15)
        
        if lat is not None:
            location_coords[location_key] = (lat, lon)
            continue
    
    # Try with state, country
    if pd.notna(row['STATE']) and pd.notna(row['COUNTRY']):
        location_info = f"{row['STATE']}, {row['COUNTRY']}"
        lat, lon = geocode_location(location_info, geolocator, delay=2.0, timeout=15)
        
        if lat is not None:
            location_coords[location_key] = (lat, lon)
            continue
    
    # Try with just country
    if pd.notna(row['COUNTRY']):
        location_info = row['COUNTRY']
        
        # Check if this country is in our hardcoded dictionary
        if location_info in hardcoded_locations:
            lat, lon = hardcoded_locations[location_info]
        else:
            lat, lon = geocode_location(location_info, geolocator, delay=2.0, timeout=15)
        
        if lat is not None:
            location_coords[location_key] = (lat, lon)
            continue

# Step 4: Map the coordinates back to the original dataframe
CGP_df['latitude'] = CGP_df['location_key'].map(lambda key: location_coords.get(key, (None, None))[0])
CGP_df['longitude'] = CGP_df['location_key'].map(lambda key: location_coords.get(key, (None, None))[1])

# Step 5: Report success and clean up
success_count = CGP_df['latitude'].notna().sum()
total_count = len(CGP_df)
print(f"Successfully geocoded {success_count}/{total_count} locations ({success_count/total_count*100:.1f}%)")

# Calculate grid cell area and convert BC units
if 'latitude' in CGP_df.columns and 'BC (g/day)' in CGP_df.columns:
    # Define grid cell area calculation
    def calculate_grid_area(lat, grid_size_degrees=1):
        """Calculate approximate grid cell area in m2 based on latitude"""
        if pd.isna(lat):
            return None
        earth_radius = 6371000  # in meters
        cell_size_lat = grid_size_degrees  # grid cell size in degrees
        cell_size_lon = grid_size_degrees  # grid cell size in degrees
        
        lat_rad = np.radians(lat)
        area = (earth_radius**2) * np.radians(cell_size_lat) * np.radians(cell_size_lon) * np.cos(lat_rad)
        return area
    
    # Calculate area for each location
    CGP_df['grid_area_m2'] = CGP_df['latitude'].apply(calculate_grid_area)
    
    # Convert BC from g/day to kg/m2
    mask = CGP_df['grid_area_m2'].notna()
    g_to_kg = 1000 #g/kg
    day_per_yr = 365 #days
    CGP_df.loc[mask, 'BC (kg/m2/year)'] = CGP_df.loc[mask, 'BC (g/day)'] * day_per_yr / g_to_kg / CGP_df.loc[mask, 'grid_area_m2']

#CGP_df = CGP_df.drop(columns=['location_key'])

Found 146 unique locations to geocode (instead of 380 individual rows)


Geocoding unique locations: 100%|██████████| 146/146 [06:06<00:00,  2.51s/it]

Successfully geocoded 380/380 locations (100.0%)


In [13]:
def read_netcdf_data(infile):
    with xr.open_dataset(infile) as ds:
        return ds



def global_annual_mean_rf(rf, sec_per_season, sec_per_year):
    # Calculate the weighted sum across seasons
    #seasonal_sum = (rf*sec_per_season).sel(season=['DJF', 'MAM', 'JJA', 'SON']).sum(dim=['season'])
    annual_sum = (rf*sec_per_season).sel(season = 'ANN')

    TJ_to_J  = 1e12

    #seasonal_mean = seasonal_sum *TJ_to_J / (area_earth * sec_per_year)
    annual_mean = annual_sum *TJ_to_J / (area_earth * sec_per_year)
    return (annual_mean.values)

def global_annual_mean_dt(dt, sec_per_season, sec_per_year):
    # Calculate the weighted sum across seasons
    #seasonal_sum = (dt * sec_per_season).sel(season=['DJF', 'MAM', 'JJA', 'SON']).sum(dim=[ 'season'])
    annual_sum = (dt * sec_per_season).sel(season = 'ANN')

    #seasonal_mean = seasonal_sum / (sec_per_year)
    annual_mean = annual_sum / (sec_per_year)

    return (annual_mean.values)


def regrid(ds, target_resolution=0.5):

    ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 90, target_resolution), {"units": "degrees_north"}),
        "lon": (["lon"], np.arange(-180, 180, target_resolution), {"units": "degrees_east"}),
    }
)

    # Create regridders
    regridder_ds = xe.Regridder(ds, ds_out, 'conservative_normed')

    ds_reg = regridder_ds(ds)

    
    return ds_out, ds_reg

def mask(ds_reg, lon_w, lon_e, lat_s, lat_n, target_resolution=1):
    
    # Create a mask for the region
    rg = np.array([[lon_w, lat_s], [lon_e, lat_s], [lon_e, lat_n], [lon_w, lat_n]])
    region = regionmask.Regions([rg], names=['emissions_loc'])
    mask_frac = region.mask_3D_frac_approx(ds_reg)

    ds_masked = ds_reg*mask_frac
    
    return ds_masked, mask_frac


def find_area(ds_masked, R=6378137):
    """ 
    Calculate the area of each grid cell from lat/lon coordinates.
    
    Parameters:
    ds (xarray.Dataset): Dataset containing 'lat' and 'lon' coordinates
    R (float): Earth's radius in km (default: 6378.1 km)
    
    Returns:
    xarray.DataArray: Grid cell areas in square km
    """
    # Ensure latitudes and longitudes are in ascending order
    ds_masked = ds_masked.sortby('lat').sortby('lon')

    # Convert to radians
    lat = np.deg2rad(ds_masked['lat'])
    lon = np.deg2rad(ds_masked['lon'])

    # Calculate differences
    dlat = lat.diff(dim='lat')
    dlon = lon.diff(dim='lon')

    # Calculate the area using the trapezoidal formula
    dx1 = R * dlon * np.cos(lat)
    dx2 = R * dlon * np.cos(lat[1:] + dlat)
    dy = R * dlat

    A = 0.5 * (dx1 + dx2) * dy
    
    return A.transpose()



In [16]:
def main(row):
    
    infile = "../../raw_data_inputs/raisanen_2022_BC_temp/praisanen-Black-carbon-radiative-forcing-and-climate-response-tool-182f7d3/BC_forcing_and_climate_response_normalized_by_emissions.nc"
    ds = read_netcdf_data(infile)
    ds = ds.rename({'nseason': 'season'})

    # Extract latitude and longitude
    lat = row['latitude']
    lon = row['longitude']
    
    # Extract emissions
    emissions = row['BC (kg/m2/year)']
    
    lmonthly_emissions = False
    emissions_annual = emissions
    emissions_monthly = None
    

    # Define seasons
    seasons = ['ANN', 'DJF', 'MAM', 'JJA', 'SON']

    ds = ds.assign_coords({'season':seasons})

    # Calculate seasonal emissions
    ndays_per_month = xr.DataArray([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], dims=['month'])
    sec_per_month = 86400 * ndays_per_month
    months_in_season = xr.DataArray([
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
    ], dims=['season', 'month'], coords = {'season':seasons})


    sec_per_season = (months_in_season * sec_per_month).sum('month')
    sec_per_year = sec_per_season.sel(season='ANN')



    # Calculate seasonal emissions
    if lmonthly_emissions:
        emissions_seasonal = (months_in_season * sec_per_month * emissions_monthly).sum(dim='month') / sec_per_season
    else:
        emissions_seasonal = xr.DataArray(np.zeros(len(seasons)), dims=['season'], coords={'season': seasons})
        emissions_seasonal.loc[{'season': 'ANN'}] = emissions_annual


    # Regrid and mask the dataset
    ds_out, ds_reg = regrid(ds)
    ds_masked_raw = ds_reg.sel(lat = lat, lon = lon, method = 'nearest')
    ds_masked = ds_masked_raw * emissions_seasonal
    

    # Calculate global annual mean radiative forcings and temperature responses
    drf_toa_out_ann = global_annual_mean_rf(ds_masked.dirsf_toa, sec_per_season, sec_per_year)
    snowrf_toa_out_ann = global_annual_mean_rf(ds_masked.snowsf_toa, sec_per_season, sec_per_year)
    dt_drf_out_ann = global_annual_mean_dt(ds_masked.dt_norm_drf, sec_per_season, sec_per_year)
    dt_snowrf_out_ann = global_annual_mean_dt(ds_masked.dt_norm_snowrf, sec_per_season, sec_per_year)
    
    #new_vars = ['drf_toa','snowrf_toa','dt_drf','dt_snowrf']
    return(pd.Series([drf_toa_out_ann, snowrf_toa_out_ann, dt_drf_out_ann, dt_snowrf_out_ann], index = new_vars))

In [19]:
new_vars = ['drf_toa','snowrf_toa','dt_drf','dt_snowrf']
CGP_df[new_vars] = CGP_df.apply(main, axis = 1)

/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/bac

In [20]:
CGP_df.to_csv(f'{utils.data_output_path}plants/BC_SE_Asia_all_financing_SEA_GAINS_Springer_plus_rad.csv')

In [299]:
## REMOVE?? 

In [296]:
def plant_shutdown_yearly(year, CGP_df, time_array, unique_id, min_year, var):
    years_running = 1
    E = np.zeros(len(time_array))
    ID_df = CGP_df.loc[CGP_df['unique_ID'] == unique_id]

    yr_offset = (ID_df['Year_of_Commission'].iloc[0] - min_year)
    test_array = np.where((time_array <= (yr_offset + years_running)) & (time_array >= yr_offset), True, False)
    E += test_array* ID_df[var].sum()
    return(E)


In [297]:

CGP_df.columns = CGP_df.columns.str.replace(' ', '_')

CGP_df = CGP_df.rename(columns = {'YEAR':'Year_of_Commission', 'EMISFACTOR.PLATTS':'CO2_weighted_capacity_1000tonsperMW'})
min_year = CGP_df['Year_of_Commission'].min()
## reduce to one country for emissions

CGP_df = CGP_df.loc[CGP_df['COUNTRY'] == country_emit]

##remove any plants that have no data on commission year
CGP_df = CGP_df.loc[CGP_df['Year_of_Commission'].dropna().index]
print('Emis data prepped and loaded')



########## Create emissions profile for each plant over our shutdown times ##########

forcing = {}
year = 1
typical_shutdown_years = 40
for unique_id in CGP_df.loc[CGP_df['BC_(g/day)'] >0]['unique_ID'].values:
    forcing[unique_id] = {}
    for var in new_vars:
        forcing[unique_id][var] = plant_shutdown_yearly(year, CGP_df, time_array, unique_id, min_year, var)

print('Forcing profiles based on weighted capacity of CO2 emissions percentiles created')


Emis data prepped and loaded
Forcing profiles based on weighted capacity of CO2 emissions percentiles created


In [298]:
forcing

{np.int64(2): {'drf_toa': array([1.17840519e-10, 1.17840519e-10, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.

In [ ]:
forcing

In [21]:
monthly = True #if false, then annual


In [22]:
df = pd.DataFrame(index = ['Latitude','Longitude','Emissions_rate'], columns = np.arange(0,12))

In [23]:
df.loc['Latitude', 0] = 60
df.loc['Latitude', 1] = 70

In [24]:
df.loc['Longitude', 0] = 22
df.loc['Longitude', 1] = 30

In [25]:
if monthly == True:
    df.loc['Emissions_rate', np.arange(0,12)] = [4.62E-13, 4.65E-13, 4.60E-13, 3.75E-13, 2.43E-13, 2.03E-13,
 2.08E-13, 2.05E-13, 2.09E-13, 3.16E-13, 4.21E-13, 4.57E-13]
    
elif monthly == False:
    df.loc['Emissions_rate', 0] = 3E-12

In [26]:
df

,0,1,2,3,4,5,6,7,8,9,10,11
Latitude,60,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Longitude,22,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Emissions_rate,4.62e-13,4.65e-13,4.6e-13,3.75e-13,2.43e-13,2.03e-13,2.08e-13,2.05e-13,2.09e-13,3.16e-13,4.21e-13,4.57e-13


In [27]:
df.to_csv('input_data.csv')